# Volco Simulation in Jupyter Notebook

This notebook demonstrates how to run the Volco simulation directly from a Jupyter notebook using the new `run_simulation` function.

In [ ]:
import json
import sys
import os
import numpy as np

# Add the parent directory to the path so we can import volco
# This only applies if this notebook is run directly from the 
# examples directory. If you run it from the root directory, you
# don't need this, nor if you've installed volco in python.
sys.path.append(os.path.abspath('..'))

from volco import run_simulation

## Example 1: Using File Paths

As with running volco from the command line or a python file, you can use file paths for settings and gcode if you prefer:

In [ ]:
# Define file paths
gcode_path = 'gcode_example.gcode'
printer_config_path = 'printer_settings.json'
sim_config_path = 'simulation_settings.json'

# Run the simulation using file paths
output = run_simulation(
    gcode_path=gcode_path,
    printer_config_path=printer_config_path,
    sim_config_path=sim_config_path
)

# Export to STL if needed
output.export_mesh_to_stl()

## Example 2: Using Python Variables

You can also pass the G-code content and configuration dictionaries directly:

In [ ]:
# Load G-code content from file
with open('gcode_example.gcode', 'r') as f:
    gcode_content = f.read()

# add new lines to the G-code 
gcode_content += '\nG0 F7200 X12.0 Y10.0 Z1.0'
gcode_content += '\nG1 F1000 X14.0 E0.133041'

# Load printer configuration from file
with open('printer_settings.json', 'r') as f:
    printer_config = json.load(f)

# Load simulation configuration from file
with open('simulation_settings.json', 'r') as f:
    sim_config = json.load(f)

# Run the simulation using Python variables
output = run_simulation(
    gcode=gcode_content,
    printer_config=printer_config,
    sim_config=sim_config
)
output.export_mesh_to_stl()

## Example 3: Creating Configuration Dictionaries Directly

You can also create the configuration dictionaries directly in Python:

In [ ]:
# Create printer configuration dictionary
custom_printer_config = {
    "nozzle_jerk_speed": 5.0, # deliberately set to a low value resulting in excess deposition at corner/start/end points 
    "extruder_jerk_speed": 5.0,
    "nozzle_acceleration": 150.0, # deliberately set to a low value resulting in excess deposition at corner/start/end points 
    "extruder_acceleration": 1200.0,
    "feedstock_filament_diameter": 1.75,
    "nozzle_diameter": 0.4
}

# Create simulation configuration dictionary
custom_sim_config = {
    "voxel_size": 0.05,
    "step_size": 0.1,
    "x_offset": 2.0,
    "y_offset": 2.0,
    "z_offset": 0.5,
    "simulation_name": "Test_volco",
    "results_folder": "Results_volco",
    "radius_increment": 0.1,
    "solver_tolerance": 0.0001,
    "x_crop": [11, 13],
    "y_crop": [9, 11],
    "z_crop": [0.0, "all"],
    "consider_acceleration": True,
    "stl_ascii": False
}

gcode_content = """M83
G0 X10 Y10 Z0.3
G1 F7200 X10.0 Y10.0 Z0.3
G1 F1000 X14.0 E0.133041
G0 F7200 X12.0 Y8.0 Z0.5
G1 F1000 Y12.0 E0.133041
G1 F7200 X10.0 Y10.0 Z0.7
G1 F1000 X12.0 E0.067
G1 F1000 X14 Y8.0 E0.1"""

# Run the simulation with custom configurations
output = run_simulation(
    gcode=gcode_content,
    printer_config=custom_printer_config,
    sim_config=custom_sim_config,
)
output.export_mesh_to_stl()

## Working with the Results

The `run_simulation` function returns the `voxel_space` and `output` objects (SimulationOutput instance), which you can use for further analysis and visualisation. The use of `output.export_mesh_to_stl()` is already demonstrated above but it may be desirable to directly output the voxel array rather than an stl for some use cases.

Visualise in a notebook:

In [ ]:
# Visualize the mesh with 'plotly' or 'trimesh' and color_scheme 'cyan_blue' or 'viridis'
fig = output.visualize_mesh(visualizer='plotly', color_scheme='cyan_blue')
fig.show()

Analyse and process data:

In [ ]:
# Access voxel space dimensions
print(f"Voxel space dimensions: {output.voxel_space.dimensions}")
# Access the cropped voxel space dimensions. E.g. calculated volume fraction of filled voxels to identify porosity
print(f"Volumetric porosity of cropped voxel space: {100*(1-np.sum(output.cropped_voxel_space/output.cropped_voxel_space.size)):.2f}%")
# check if the voxel space has entirely unfilled columns (top-down porosity) and identify top-down porosity
# get a 2d top-down representation of the voxel space with 1s if there are any filled voxels in the column
top_down_voxel_space = np.sum(output.cropped_voxel_space, axis=2)
# rotation is required to get the correct orientation for x and y
top_down_voxel_space = np.rot90(top_down_voxel_space, k=1, axes=(0, 1))
# convert from sums per column to 1s to get a binary matrix
top_down_voxel_space[top_down_voxel_space > 0] = 1
print(f"Top-down porosity: {100*(np.sum(top_down_voxel_space == 0)/top_down_voxel_space.size):.2f}%")
# create a 2D plot of the top-down view
import plotly.express as px
fig = px.imshow(top_down_voxel_space, color_continuous_scale='gray', aspect='equal')
fig.update_layout(xaxis_title='X-axis (columns)',yaxis_title='Y-axis (rows)',coloraxis_showscale=False)
fig.show()